In [1]:
import pandas as pd
import re
import datetime
from datetime import datetime
import pandasql
from pandasql import sqldf
import duckdb
import plotly.express as px
import pandoc

In [3]:
visits = pd.read_csv('/Users/jullia/Downloads/Задания тендер/MFPlus_visits.csv',sep=';')

In [4]:
visits.head()

ym:s:dateTime         ym:s:visitID        ym:s:clientID  \
0  2022-05-01 22:30:37  2305985862102679718  1648991419730583177   
1  2022-04-28 22:31:57  2238059103780601873  1651082186327730631   
2  2022-04-22 19:28:11  2099273109514748205   163851657390912688   
3  2022-04-19 10:58:55  2023315303962509512  1650311540470343111   
4  2022-05-09 04:31:18  2470203429493932112  1651822039339341135   

   ym:s:visitDuration  
0                   7  
1                   7  
2                   0  
3                  15  
4                   0

In [5]:
visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126131 entries, 0 to 126130
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   ym:s:dateTime       126131 non-null  object
 1   ym:s:visitID        126131 non-null  int64 
 2   ym:s:clientID       126131 non-null  uint64
 3   ym:s:visitDuration  126131 non-null  int64 
dtypes: int64(2), object(1), uint64(1)
memory usage: 3.8+ MB


In [6]:
#renaming columns
visit_cols = ['date','visit_id','client_id','visit_duration']

In [7]:
visits.columns = visit_cols

In [8]:
visits.head()

date             visit_id            client_id  \
0  2022-05-01 22:30:37  2305985862102679718  1648991419730583177   
1  2022-04-28 22:31:57  2238059103780601873  1651082186327730631   
2  2022-04-22 19:28:11  2099273109514748205   163851657390912688   
3  2022-04-19 10:58:55  2023315303962509512  1650311540470343111   
4  2022-05-09 04:31:18  2470203429493932112  1651822039339341135   

   visit_duration  
0               7  
1               7  
2               0  
3              15  
4               0

In [9]:
# определяем начало сессии, приводим к формату даты-времени
visits['visit_start'] = pd.to_datetime(visits['date'])

In [10]:
hits = pd.read_csv('/Users/jullia/Downloads/Задания тендер/MFPlus_hits.csv', sep=';')


In [11]:
hits.head()

ym:pv:dateTime        ym:pv:watchID        ym:pv:clientID  \
0  2022-05-15 22:51:06  2623397282019803354  16138276891048456082   
1  2022-05-15 22:51:12  2623398959716761898  16138276891048456082   
2  2022-05-15 22:51:12  2623398977377402942  16138276891048456082   
3  2022-05-15 22:51:20  2623400872067203452  16138276891048456082   
4  2022-05-15 22:51:20  2623400887705403727  16138276891048456082   

                                     ym:pv:URL  \
0                     https://plus.megafon.ru/   
1                     https://plus.megafon.ru/   
2  goal://plus.megafon.ru/MegafonPlusScroll_20   
3                     https://plus.megafon.ru/   
4  goal://plus.megafon.ru/MegafonPlusScroll_40   

              ym:pv:parsedParamsKey2                  ym:pv:parsedParamsKey3  \
0                                 []                                      []   
1  ['plus-web [prod] / NAVIGATIONS']  ['useNavigations / [МФ+] / Scroll / ']   
2                                 []                                      []   
3  ['plus-web [prod] / NAVIGATIONS']  ['useNavigations / [МФ+] / Scroll / ']   
4                                 []                                      []   

  ym:pv:parsedParamsKey4 ym:pv:parsedParamsKey5  
0                     []                     []  
1                 ['20']          ['d-v429-e1']  
2                     []                     []  
3                 ['40']          ['d-v429-e1']  
4                     []                     []

In [12]:
# переименовываем столбцы
hits_columns = ['date','watch_id','client_id','url','ParamsKey2','ParamsKey3','ParamsKey4','ParamsKey5']

In [13]:
hits.columns = hits_columns

In [14]:
hits.head(5)

date             watch_id             client_id  \
0  2022-05-15 22:51:06  2623397282019803354  16138276891048456082   
1  2022-05-15 22:51:12  2623398959716761898  16138276891048456082   
2  2022-05-15 22:51:12  2623398977377402942  16138276891048456082   
3  2022-05-15 22:51:20  2623400872067203452  16138276891048456082   
4  2022-05-15 22:51:20  2623400887705403727  16138276891048456082   

                                           url  \
0                     https://plus.megafon.ru/   
1                     https://plus.megafon.ru/   
2  goal://plus.megafon.ru/MegafonPlusScroll_20   
3                     https://plus.megafon.ru/   
4  goal://plus.megafon.ru/MegafonPlusScroll_40   

                          ParamsKey2                              ParamsKey3  \
0                                 []                                      []   
1  ['plus-web [prod] / NAVIGATIONS']  ['useNavigations / [МФ+] / Scroll / ']   
2                                 []                                      []   
3  ['plus-web [prod] / NAVIGATIONS']  ['useNavigations / [МФ+] / Scroll / ']   
4                                 []                                      []   

  ParamsKey4     ParamsKey5  
0         []             []  
1     ['20']  ['d-v429-e1']  
2         []             []  
3     ['40']  ['d-v429-e1']  
4         []             []

In [15]:
#добавляем время хита в формате даты-времени
hits['hit_time'] = pd.to_datetime(hits['date'])

In [16]:
hits['watch_id'] = hits['watch_id'].astype('str')

In [17]:
# определяем окончание сессии
visits['visit_end'] = visits['visit_start'] + pd.to_timedelta(visits['visit_duration'], unit='m')

In [18]:
visits

date             visit_id            client_id  \
0       2022-05-01 22:30:37  2305985862102679718  1648991419730583177   
1       2022-04-28 22:31:57  2238059103780601873  1651082186327730631   
2       2022-04-22 19:28:11  2099273109514748205   163851657390912688   
3       2022-04-19 10:58:55  2023315303962509512  1650311540470343111   
4       2022-05-09 04:31:18  2470203429493932112  1651822039339341135   
...                     ...                  ...                  ...   
126126  2022-05-08 08:55:01  2451702035276890223  1643485052686289873   
126127  2022-05-12 17:50:52  2550727300977262630  1626677638121299271   
126128  2022-05-07 19:40:26  2439204455185645824  1619204793497286942   
126129  2022-05-17 06:39:19  2653410888589771079  1652758757191003836   
126130  2022-05-11 11:08:45  2521753307715731674  1652014285507387565   

        visit_duration         visit_start           visit_end  
0                    7 2022-05-01 22:30:37 2022-05-01 22:37:37  
1                    7 2022-04-28 22:31:57 2022-04-28 22:38:57  
2                    0 2022-04-22 19:28:11 2022-04-22 19:28:11  
3                   15 2022-04-19 10:58:55 2022-04-19 11:13:55  
4                    0 2022-05-09 04:31:18 2022-05-09 04:31:18  
...                ...                 ...                 ...  
126126               9 2022-05-08 08:55:01 2022-05-08 09:04:01  
126127              15 2022-05-12 17:50:52 2022-05-12 18:05:52  
126128              25 2022-05-07 19:40:26 2022-05-07 20:05:26  
126129              49 2022-05-17 06:39:19 2022-05-17 07:28:19  
126130              15 2022-05-11 11:08:45 2022-05-11 11:23:45  

[126131 rows x 6 columns]

In [19]:
#преобразуем шаги для удобства обработки
def work_with_string(el):
    return "_".join(re.sub(r'[\[\]\/\']',"",el).replace("  "," ").strip().split(" "))

In [20]:
cols = ['ParamsKey2', 'ParamsKey3', 'ParamsKey4', 'ParamsKey5']

In [21]:
for col  in cols:
    hits[col] = hits[col].apply(lambda x: work_with_string(x))

In [22]:
hits.head()

date             watch_id             client_id  \
0  2022-05-15 22:51:06  2623397282019803354  16138276891048456082   
1  2022-05-15 22:51:12  2623398959716761898  16138276891048456082   
2  2022-05-15 22:51:12  2623398977377402942  16138276891048456082   
3  2022-05-15 22:51:20  2623400872067203452  16138276891048456082   
4  2022-05-15 22:51:20  2623400887705403727  16138276891048456082   

                                           url                 ParamsKey2  \
0                     https://plus.megafon.ru/                              
1                     https://plus.megafon.ru/  plus-web_prod_NAVIGATIONS   
2  goal://plus.megafon.ru/MegafonPlusScroll_20                              
3                     https://plus.megafon.ru/  plus-web_prod_NAVIGATIONS   
4  goal://plus.megafon.ru/MegafonPlusScroll_40                              

                  ParamsKey3 ParamsKey4 ParamsKey5            hit_time  
0                                                  2022-05-15 22:51:06  
1  useNavigations_МФ+_Scroll         20  d-v429-e1 2022-05-15 22:51:12  
2                                                  2022-05-15 22:51:12  
3  useNavigations_МФ+_Scroll         40  d-v429-e1 2022-05-15 22:51:20  
4                                                  2022-05-15 22:51:20

In [23]:
#импортируем словарь шагов
steps = pd.read_csv('/Users/jullia/Downloads/Копия Описание параметров событий_МФ Плюс.xlsx - События воронки (1).csv', nrows=28)

In [24]:
steps = steps.fillna("")

In [25]:
for col  in cols:
    steps[col] = steps[col].apply(lambda x: work_with_string(x))

In [26]:
steps = steps.rename(columns={'Шаги воронки':'step'})

In [27]:
steps.tail()

step                                   Описание событий  \
23     6  Успешный ввод кода - авторизация в форме регис...   
24     6  Успешный ввод кода - авторизация в форме регис...   
25     7                                Отключение подписки   
26     7                                Отключение подписки   
27     7                                Отключение подписки   

                   ParamsKey2                                 ParamsKey3  \
23  plus-web_prod_CONVERSIONS              conversionStep_МФ+_sendInput2   
24  plus-web_prod_CONVERSIONS                conversionStepМФ+sendInput2   
25  plus-web_prod_CONVERSIONS       conversionStep_МФ+_submitTransaction   
26  plus-web_prod_CONVERSIONS  conversionStep_МФ+_submitTransaction_auth   
27  plus-web_prod_CONVERSIONS       conversionStep_МФ+_submitTransaction   

                            ParamsKey4  ParamsKey5  
23        Продолжить_Введен_код_из_смс  d-v412-e18  
24        Продолжить_Введен_код_из_смс  d-v412-e18  
25  Абонент_Отключение_подписки_Готово  d-v412-e33  
26  Абонент_Отключение_подписки_Готово  d-v412-e33  
27  Абонент_Отключение_подписки_Готово  d-v412-e33

In [28]:
#объединяем шаги с хитами, каждому хиту ставим во соответствие шег
hits_with_steps = hits.merge(steps,how='left', on=['ParamsKey2','ParamsKey3','ParamsKey4','ParamsKey5'] )

In [29]:

hits_with_steps = hits_with_steps.drop('Описание событий',axis=1)


In [30]:
hits_with_steps.head()

date             watch_id             client_id  \
0  2022-05-15 22:51:06  2623397282019803354  16138276891048456082   
1  2022-05-15 22:51:12  2623398959716761898  16138276891048456082   
2  2022-05-15 22:51:12  2623398977377402942  16138276891048456082   
3  2022-05-15 22:51:20  2623400872067203452  16138276891048456082   
4  2022-05-15 22:51:20  2623400887705403727  16138276891048456082   

                                           url                 ParamsKey2  \
0                     https://plus.megafon.ru/                              
1                     https://plus.megafon.ru/  plus-web_prod_NAVIGATIONS   
2  goal://plus.megafon.ru/MegafonPlusScroll_20                              
3                     https://plus.megafon.ru/  plus-web_prod_NAVIGATIONS   
4  goal://plus.megafon.ru/MegafonPlusScroll_40                              

                  ParamsKey3 ParamsKey4 ParamsKey5            hit_time  step  
0                                                  2022-05-15 22:51:06   NaN  
1  useNavigations_МФ+_Scroll         20  d-v429-e1 2022-05-15 22:51:12   NaN  
2                                                  2022-05-15 22:51:12   NaN  
3  useNavigations_МФ+_Scroll         40  d-v429-e1 2022-05-15 22:51:20   NaN  
4                                                  2022-05-15 22:51:20   NaN

In [31]:
#считаем количество посещений 1 странцицы
first_page = hits_with_steps.query('url=="https://plus.megafon.ru/"')['client_id'].count()

In [32]:
#удаляем хиты, у которых нет шагов воронки
hits_with_steps = hits_with_steps.dropna()

In [33]:
hits_with_steps = hits_with_steps[['watch_id', 'client_id','step','hit_time','url']]

In [34]:
hits_with_steps.head()

watch_id            client_id  step            hit_time  \
39  2618775777306476874  1652626599649969589   2.0 2022-05-15 17:57:16   
43  2618776365009207582  1652626599649969589   3.0 2022-05-15 17:57:19   
44  2618779816291729734  1652626599649969589   4.0 2022-05-15 17:57:32   
47  2618787341541310790  1652626599649969589   5.0 2022-05-15 17:58:00   
48  2618787341541310790  1652626599649969589   6.0 2022-05-15 17:58:00   

                                                  url  
39  https://plus.megafon.ru/?advcake_params=59e9a5...  
43  https://account.megafon.ru/auth/realms/Subscri...  
44  https://account.megafon.ru/auth/realms/Subscri...  
47  https://account.megafon.ru/auth/realms/Subscri...  
48  https://account.megafon.ru/auth/realms/Subscri...

In [35]:
hits_with_steps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111818 entries, 39 to 995746
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   watch_id   111818 non-null  object        
 1   client_id  111818 non-null  uint64        
 2   step       111818 non-null  float64       
 3   hit_time   111818 non-null  datetime64[ns]
 4   url        111818 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(2), uint64(1)
memory usage: 5.1+ MB


In [36]:
visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126131 entries, 0 to 126130
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            126131 non-null  object        
 1   visit_id        126131 non-null  int64         
 2   client_id       126131 non-null  uint64        
 3   visit_duration  126131 non-null  int64         
 4   visit_start     126131 non-null  datetime64[ns]
 5   visit_end       126131 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(1), uint64(1)
memory usage: 5.8+ MB


In [37]:
visits['client_id'] = visits['client_id'].astype('str')

In [38]:
#удаляем дефис в начале некоторых clientid
def remove_unknown_simbols(el):
    return (re.sub(r'-',"",el).replace("  "," ").strip())

In [39]:
visits['client_id'] = visits['client_id'].apply(lambda x: remove_unknown_simbols(x))

In [40]:
visits['visit_duration'] = visits['visit_duration'].astype('int')

In [41]:
visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126131 entries, 0 to 126130
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            126131 non-null  object        
 1   visit_id        126131 non-null  int64         
 2   client_id       126131 non-null  object        
 3   visit_duration  126131 non-null  int64         
 4   visit_start     126131 non-null  datetime64[ns]
 5   visit_end       126131 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(2)
memory usage: 5.8+ MB


In [42]:
hits['client_id'] = hits['client_id'].astype('str')
hits['watch_id'] = hits['watch_id'].astype('str')

In [43]:
query = """ 
select 
    v.date, 
    cast(v.visit_id as string) as visit_id,
    cast(v.client_id as string) as client_id,
    v.visit_duration,
    v.visit_start,
    v.visit_end , 
    cast(h.watch_id as string) as watch_id,
    h.hit_time,
    h.client_id,
    h.step,
    h.url
from visits v left join hits_with_steps h
on v.client_id = h.client_id and  visit_start <= h.hit_time and v.visit_end >= h.hit_time
"""


In [44]:
#объединяем сессии и хиты
data = duckdb.query(query).df()

In [45]:
data = data.dropna()

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122909 entries, 0 to 230896
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            122909 non-null  object        
 1   visit_id        122909 non-null  object        
 2   client_id       122909 non-null  object        
 3   visit_duration  122909 non-null  int64         
 4   visit_start     122909 non-null  datetime64[ns]
 5   visit_end       122909 non-null  datetime64[ns]
 6   watch_id        122909 non-null  object        
 7   hit_time        122909 non-null  datetime64[ns]
 8   client_id_2     122909 non-null  float64       
 9   step            122909 non-null  float64       
 10  url             122909 non-null  object        
dtypes: datetime64[ns](3), float64(2), int64(1), object(5)
memory usage: 11.3+ MB


In [47]:
data['step'] = data['step'].astype('int')

In [48]:
data.tail()

date             visit_id            client_id  \
230892  2022-05-05 10:26:06  2385187070672109764  1649011726966690886   
230893  2022-05-05 10:26:06  2385187070672109764  1649011726966690886   
230894  2022-05-05 10:26:06  2385187070672109764  1649011726966690886   
230895  2022-05-05 10:26:06  2385187070672109764  1649011726966690886   
230896  2022-05-05 10:26:06  2385187070672109764  1649011726966690886   

        visit_duration         visit_start           visit_end  \
230892             115 2022-05-05 10:26:06 2022-05-05 12:21:06   
230893             115 2022-05-05 10:26:06 2022-05-05 12:21:06   
230894             115 2022-05-05 10:26:06 2022-05-05 12:21:06   
230895             115 2022-05-05 10:26:06 2022-05-05 12:21:06   
230896             115 2022-05-05 10:26:06 2022-05-05 12:21:06   

                   watch_id            hit_time   client_id_2  step  \
230892  2385208943041577156 2022-05-05 10:27:30  1.649012e+18     5   
230893  2385203691447910596 2022-05-05 10:27:09  1.649012e+18     4   
230894  2385191393437941918 2022-05-05 10:26:23  1.649012e+18     4   
230895  2385191003138031804 2022-05-05 10:26:21  1.649012e+18     3   
230896  2385189472422854993 2022-05-05 10:26:15  1.649012e+18     2   

                                                      url  
230892  https://account.megafon.ru/auth/realms/Subscri...  
230893  https://account.megafon.ru/auth/realms/Subscri...  
230894  https://account.megafon.ru/auth/realms/Subscri...  
230895  https://account.megafon.ru/auth/realms/Subscri...  
230896                           https://plus.megafon.ru/

In [49]:
#определяем последовательность шагов для каждой сессии
data_reduced = data.sort_values(by='hit_time')[['visit_id','step']].groupby('visit_id',as_index=False).agg(lambda x : list(x))

In [50]:
data_reduced.head(5)

visit_id                                               step
0  2023521183830376547  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 6, ...
1  2025034091373527214                                             [2, 2]
2  2025360039948058723  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 6, ...
3  2026107441266819171                  [2, 2, 2, 2, 3, 3, 6, 6, 2, 2, 2]
4  2026958335234277699                                          [2, 2, 2]

In [51]:
# опредляем финальный шаг сессии, если в сесссии есть все шаги до шага n - она относится к шагу n
def final_step(el):
    if (2 in el) and (3 not in el and 4 not in el and 5 not in el and 6 not in el and 7 not in el):
        return 2
    elif (2 in el and 3 in el) and (4 not in el and 5 not in el and 6 not in el and 7 not in el):
        return 3
    elif (2 in el and 3 in el and 4 in el) and (5 not in el and 6 not in el and 7 not in el):
        return 4
    elif (2 in el and 3 in el and 4 in el and 5 in el) and (6 not in el and 7 not in el):
        return 5
    elif (2 in el and 3 in el and 4 in el and 5 in el and 6 in el) and (7 not in el):
        return 6
    elif (2 in el and 3 in el and 4 in el and 5 in el and 6 in el and 7 in el):
        return 7
    else:
        return 0


In [52]:
data_reduced.head()

visit_id                                               step
0  2023521183830376547  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 6, ...
1  2025034091373527214                                             [2, 2]
2  2025360039948058723  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 6, ...
3  2026107441266819171                  [2, 2, 2, 2, 3, 3, 6, 6, 2, 2, 2]
4  2026958335234277699                                          [2, 2, 2]

In [53]:
data_reduced['final_step'] = data_reduced['step'].map(lambda x: final_step(x))

In [54]:
data_reduced.tail(15)

visit_id                                               step  \
17982  2714137303219175787                                                [3]   
17983  2714186115679519116                                    [2, 3, 4, 6, 5]   
17984  2714295532039962696                                    [2, 3, 4, 5, 6]   
17985  2714341643749425765  [2, 3, 6, 5, 3, 3, 6, 6, 3, 3, 6, 6, 3, 3, 3, ...   
17986  2714508551042367754                                             [2, 3]   
17987  2714520323978952877                                             [2, 3]   
17988  2714531766642671716               [2, 3, 2, 2, 3, 4, 4, 6, 5, 5, 6, 2]   
17989  2714546244388913346                                    [2, 3, 4, 6, 5]   
17990  2714563367654916385         [3, 4, 5, 6, 6, 5, 3, 4, 5, 6, 5, 6, 6, 5]   
17991  2714618359076618277  [2, 3, 4, 3, 2, 3, 4, 6, 5, 6, 5, 3, 2, 3, 4, ...   
17992  2714633647295561752                                    [2, 3, 4, 6, 5]   
17993  2714659291614413163                                             [2, 3]   
17994  2714705885288988822                                    [2, 3, 4, 5, 6]   
17995  2714985692970876986                                             [2, 3]   
17996  2715000246710305095               [2, 3, 6, 5, 3, 3, 3, 3, 3, 3, 3, 3]   

       final_step  
17982           0  
17983           6  
17984           6  
17985           0  
17986           3  
17987           3  
17988           6  
17989           6  
17990           0  
17991           6  
17992           6  
17993           3  
17994           6  
17995           3  
17996           0

In [55]:
#группируем полученные данные по финальным шагам
grouped = data_reduced.groupby('final_step', as_index=False).count()

In [56]:
#добавляем первый шаг - просмотр начальной страницы
grouped.loc[len(grouped.index)] = [1, first_page, first_page]

In [57]:
#удаляем нулевой шаг - когда финальный шаг не может быть определен
grouped = grouped.drop(0,axis=0)

In [58]:
final_df = grouped.sort_values(by='final_step')

In [59]:
final_df

final_step  visit_id    step
6           1    418619  418619
1           2      2424    2424
2           3      3744    3744
3           4       543     543
4           6      7111    7111
5           7       804     804

In [60]:
from plotly import graph_objects as go

fig = go.Figure(go.Funnel(
    y = ["First page", "Click enter", "Registration", "Phone num input","Code accepted","Unsubscribe"],
    x = final_df['step']))

fig.show()

In [63]:
data = dict(
    number= [2424,3744,543,7111,804],
    stage=['2','3','4','6','7'])
fig = px.funnel(data, x='number', y='stage')
fig.show()

In [64]:
data_reduced.query('5 in step & ~(6 in step)')

visit_id                                    2023521183830376547
step          [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 6, ...
final_step                                                    0
Name: 0, dtype: object

Нет таких сессий, в которыйх бы был 5 шаг  и не было б шестого